## Librerias

In [1]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)


## Conectar a AmigoCloud

In [3]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [4]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

## Datos de crédito

In [5]:
semana = 'S14'

In [6]:
query = {'query': 'select insp.canhero, prop.propiedad, lote.lote, gal.s3_filename, gal.amigocloud_url foto\
         from dataset_307560 insp\
         inner join dataset_307562 prop on insp.amigo_id=prop.insp_ref_prop\
         inner join dataset_307563 lote on prop.amigo_id=lote.prop_ref_lote\
         inner join gallery_46845 gal on lote.amigo_id=gal.source_amigo_id\
         where semana_1ra = \'{sem}\' order by canhero'.format(sem=semana)}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query)
data = select['data']
#credito = collections.namedtuple("credito", credito_data.keys())(*credito_data.values())
print('end')

end


In [7]:
data

[{'canhero': '1979 / CLAROS CADIMA EDMUNDO ',
  'propiedad': '92 / SAN JOSE--CLAROS CADIMA',
  'lote': 'L11',
  's3_filename': 'WhatsApp Image 2023-04-06 at 16.04.35.jpeg',
  'foto': 'app.amigocloud.com/api/v1/related_tables/46845/files/dbdd35c5e3aa4da9b08f547923bce887/27fee092668c4a0b807924cc6b979c6f/WhatsApp%20Image%202023-04-06%20at%2016.04.35.jpeg'},
 {'canhero': '1979 / CLAROS CADIMA EDMUNDO ',
  'propiedad': '92 / SAN JOSE--CLAROS CADIMA',
  'lote': 'L17',
  's3_filename': 'WhatsApp Image 2023-04-06 at 16.04.35 (2).jpeg',
  'foto': 'app.amigocloud.com/api/v1/related_tables/46845/files/f234b88401d44b55a496e0ad4cb8d50d/0840b25c0bd94b1d8cb8f0ecd2f54b52/WhatsApp%20Image%202023-04-06%20at%2016.04.35%20(2).jpeg'},
 {'canhero': '2522 / CARO SONIA GENOVEVA CLAROS DE ',
  'propiedad': '92 / SAN JOSE--CLAROS CADIMA',
  'lote': 'L8.2',
  's3_filename': 'WhatsApp Image 2023-04-06 at 16.04.36 (1).jpeg',
  'foto': 'app.amigocloud.com/api/v1/related_tables/46845/files/40e0386471e74ae891d5725f64

In [10]:
from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm

import requests

doc = DocxTemplate(ruta + "/templates/tpl_1ra_fotos.docx")

#descargar fotos y generar lista InlineImage
lista_fotos_inline = []
for foto in data:
    response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
    file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
    file.write(response.content)
    file.close()
    lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor=ruta + '/fotos/' + foto['s3_filename'], width=Mm(120)), 'canhero': foto['canhero']})
    print(foto['s3_filename'])

context = {'fotos':lista_fotos_inline}

doc.render(context)
file_name = 'SEMANA - ' + semana
doc.save(ruta + '/_' + file_name + '.docx')

WhatsApp Image 2023-04-06 at 16.04.35.jpeg
WhatsApp Image 2023-04-06 at 16.04.35 (2).jpeg
WhatsApp Image 2023-04-06 at 16.04.36 (1).jpeg
